In [1]:
import math
import random
from tabulate import tabulate

def sambrero_funk(x1, x2):
    result = (1 - pow(math.sin(math.sqrt(pow(x1,2) + pow(x2,2))),2)/(1 + 0.001 * (pow(x1,2) + pow(x2,2)))) + 100
    return result

def gen_population(population_size, bottom_line, upper_limit):
    return [[random.uniform(bottom_line, upper_limit), random.uniform(bottom_line, upper_limit)] for _ in range(population_size)]

def gen_population_kod_Grey(population_size, bottom_line, upper_limit):
    return [[random.randint(bottom_line, upper_limit), random.randint(bottom_line, upper_limit)] for _ in range(population_size)]

def display_population(population):
    # Выводит популяцию в виде красивой таблицы.
    # 
    # :param population: Список особей (каждая особь представлена в виде списка значений)

    headers = [f"Хромосома {i + 1}" for i in range(len(population[0]))]
    table = tabulate(population, headers=headers, tablefmt="fancy_grid")
    print(table)

In [2]:
# принимает вещественное число и возвращает его код Грея
def decimal_to_gray_binary(decimal):
    gray = decimal ^ (decimal >> 1)
    binary_gray = bin(gray)[2:]
    return binary_gray

# принимает код Грея в вещественное число
def gray_binary_to_decimal(gray_binary):
    gray_decimal = int(gray_binary, 2)
    decimal = 0
    while gray_decimal:
        decimal ^= gray_decimal
        gray_decimal >>= 1
    return decimal

# принимает популяцию вещественных чисел и возвращает ее в коде Грея
def population_to_gray(decimal_population):
    gray_population = []

    for decimal_individual in decimal_population:
        gray_individual = [decimal_to_gray_binary(gene) for gene in decimal_individual]
        gray_population.append(gray_individual)
    
    return gray_population

# принимает популяцию чисел в коде Грея и возвращает ее в виде целых чисел
def population_to_decimal(gray_population):
    decimal_population = []

    for gray_individual in gray_population:
        decimal_individual = [gray_binary_to_decimal(gene) for gene in gray_individual]
        decimal_population.append(decimal_individual)

    return decimal_population

Функции отбора родителей

In [3]:
def tournament_otbor(population, tournament_size):
    # Выполняет турнирный отбор родителей из популяции.
    # 
    # :param population: Список особей (родителей)
    # :param tournament_size: Размер турнира, количество участников турнира
    # :return: Особь-победитель турнира (родитель)
    
    if len(population) < tournament_size:
        raise ValueError("Размер турнира не может быть больше размера популяции")

    # Выбираем случайные участников турнира
    tournament_participants = random.sample(population, tournament_size)

    # Сортируем участников турнира по их приспособленности
    tournament_participants.sort(key=lambda x: sambrero_funk(x[0],x[1]), reverse=True)

    # Возвращаем особь-победителя турнира
    return tournament_participants[0]

In [4]:
def euclidean_distance(v1, v2):
    # Вычисляет евклидово расстояние между двумя векторами.
    # 
    # :param v1: Первый вектор (список значений)
    # :param v2: Второй вектор (список значений)
    # :return: Евклидово расстояние между векторами
    
    return math.sqrt(sum((x - y)**2 for x, y in zip(v1, v2)))

def inbreeding_with_distance(parent, population):
    # Выполняет инбридинг между первым родителем и вторым родителем,
    # выбранным из популяции ближайшим к первому по евклидову расстоянию.
    # 
    # :param parent: Первый родитель (список значений)
    # :param population: Список особей в популяции (каждая особь - список значений)
    # :return: Список, содержащий двух потомков после инбридинга
    
    # Исключаем родителя из популяции
    available_parents = [ind for ind in population if ind != parent]
    
    closest_parent = min(available_parents, key=lambda ind: euclidean_distance(parent, ind))
    
    offspring = [(gene_parent + gene_closest) / 2 for gene_parent, gene_closest in zip(parent, closest_parent)]

    return offspring

def inbreeding_with_distance_for_grey(parent, population):
    # Выполняет инбридинг между первым родителем и вторым родителем,
    # выбранным из популяции ближайшим к первому по евклидову расстоянию.
    # 
    # :param parent: Первый родитель (список значений)
    # :param population: Список особей в популяции (каждая особь - список значений)
    # :return: Список, содержащий двух потомков после инбридинга
    
    # Исключаем родителя из популяции
    available_parents = [ind for ind in population if ind != parent]
    
    closest_parent = min(available_parents, key=lambda ind: euclidean_distance(parent, ind))
    
    # Выполняем инбридинг
    offspring = [round((gene_parent + gene_closest) / 2 + random.choice([-0.5, 0.5])) for gene_parent, gene_closest in zip(parent, closest_parent)]
    
    # Убеждаемся, что значения генов находятся в пределах [0, 10]
    offspring = [max(0, min(10, gene)) for gene in offspring]

    return offspring

Функция проверки сходимости

In [5]:
def convergence_check(population, convergence_threshold):
    
    # Проверяет схождение популяции, основываясь на среднем расстоянии между хромосомами.
    # 
    # :param population: Популяция
    # :param convergence_threshold: Порог схождения, расстояние, ниже которого популяция считается сойденной
    # :return: True, если популяция сошлась, False в противном случае
    
    if len(population) < 2:
        return False

    total_distance = 0

    for i in range(len(population) - 1):
        for j in range(i + 1, len(population)):
            # Рассчитываем евклидово расстояние между хромосомами
            distance = ((population[i][0] - population[j][0])**2 + (population[i][1] - population[j][1])**2)**0.5
            total_distance += distance

    # Среднее расстояние между хромосомами
    average_distance = total_distance / (len(population) * (len(population) - 1) / 2)

    # print(average_distance)

    # Проверка схождения
    return average_distance < convergence_threshold

Функции рекомбинации

In [6]:
def discrete_recombination(parent1, parent2):
    
    # Выполняет дискретную рекомбинацию для вещественных чисел между двумя родителями.
    # 
    # :param parent1: Первый родитель (список вещественных чисел)
    # :param parent2: Второй родитель (список вещественных чисел)
    # :return: Offspring Потомок (новая особь после рекомбинации)
    
    if len(parent1) != len(parent2):
        raise ValueError("Размеры родителей должны быть одинаковыми")

    individual_length = len(parent1)
    split_point = random.randint(1, individual_length - 1)

    # Выбираем значения от первого родителя до точки раздела
    offspring1 = parent1[:split_point]

    # Добавляем значения от второго родителя после точки раздела
    offspring1.extend(parent2[split_point:])
    
    # Выбираем значения от первого родителя до точки раздела
    offspring2 = parent2[:split_point]

    # Добавляем значения от второго родителя после точки раздела
    offspring2.extend(parent1[split_point:])

    return [offspring1, offspring2]

In [7]:
def linear_recombination(parent1, parent2):
    # Выполняет линейную рекомбинацию между двумя родителями.
    # 
    # :param parent1: Первый родитель (список значений)
    # :param parent2: Второй родитель (список значений)
    # :return: Список, содержащий двух потомков после линейной рекомбинации
    
    alpha = random.uniform(0, 1)  # Весовой коэффициент для рекомбинации

    # Линейная рекомбинация: взвешенная сумма генов
    child1 = [alpha * gene1 + (1 - alpha) * gene2 for gene1, gene2 in zip(parent1, parent2)]
    child2 = [alpha * gene2 + (1 - alpha) * gene1 for gene1, gene2 in zip(parent1, parent2)]

    return [child1, child2]

Функции рекомбинации для двоичного кода Грея

In [8]:
def expand_genes(genes):
    # Преобразуем каждый ген в строку длиной 4, дополняя нулями слева при необходимости
    expanded_genes = [format(int(gene, 2), '04b') for gene in genes]
    return expanded_genes

def gray_crossover(parent1, parent2):
    # Выбираем случайное место для кроссинговера
    crossover_point = random.randint(1, len(parent1) - 1)

    # Выполняем кроссинговер для каждого гена
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return [child1, child2]

In [9]:
def gray_binary_shuffle_crossover(parent1, parent2):
    # Перемешиваем гены обоих родителей случайным образом
    child1 = ''.join(random.sample(''.join(parent1), len(''.join(parent1))))
    child2 = ''.join(random.sample(''.join(parent2), len(''.join(parent2))))

    # Разбиваем на два признака
    child1_features = [child1[:2], child1[2:]]
    child2_features = [child2[:2], child2[2:]]

    for i in range(2):
        if not child1_features[i].strip():
            child1_features[i] = '0'
        if not child2_features[i].strip():
            child2_features[i] = '0'
            
    for i in range(2):
        if len(child1_features[i]) > 4:
            child1_features[i] = child1_features[i][:4]
        if len(child2_features[i]) > 4:
            child2_features[i] = child2_features[i][:4]

    return [child1_features, child2_features]

Функции мутации

In [10]:
def mutation(individual, mutation_rate):
    # Выполняет мутацию с заданной вероятностью для каждого элемента особи.
    # 
    # :param individual: Особь, которую нужно мутировать (список значений)
    # :param mutation_rate: Вероятность мутации для каждого элемента (от 0 до 1)
    # :return: Мутированная особь
    
    mutated_individual = []

    for gene in individual:
        # Мутируем каждый ген с фиксированной вероятностью
        if random.random() < mutation_rate:
            # Мутируем ген (вы можете использовать другую стратегию мутации)
            mutated_gene = gene + random.uniform(-1, 1)
            # Убеждаемся, что мутированный ген находится в пределах [-10, 10]
            mutated_gene = max(-10, min(10, mutated_gene))
        else:
            mutated_gene = gene

        mutated_individual.append(mutated_gene)

    return mutated_individual

Функция мутации для двоичного кода Грея

In [11]:
def gray_binary_mutation(individual, mutation_rate):
    mutated_individual = []

    for gene in individual:
        mutated_gene = ""
        for bit in gene:
            # Мутируем каждый бит с фиксированной вероятностью
            if random.random() < mutation_rate:
                mutated_gene += '0' if bit == '1' else '1'
            else:
                mutated_gene += bit

        mutated_gene = gray_binary_to_decimal(mutated_gene)
        mutated_gene = max(0, min(10, mutated_gene))
        mutated_gene = decimal_to_gray_binary(mutated_gene)
        mutated_individual.append(mutated_gene)

    return mutated_individual

Функции селекции

In [12]:
def proportional_selection(population, num_parents):
    # Выполняет пропорциональный отбор особей из популяции.
    # 
    # :param population: Список особей (каждая особь представлена в виде списка значений)
    # :param количество_родителей: Количество родителей, которое нужно выбрать
    # :return: Список выбранных родителей
    
    fitness_values = [sambrero_funk(individual[0],individual[1]) for individual in population]
    total_fitness = sum(fitness_values)
    
    # Рулеточный выбор родителей
    selected_parents = []
    for _ in range(num_parents):
        roulette_wheel = random.uniform(0, total_fitness)
        current_sum = 0

        for i, fitness in enumerate(fitness_values):
            current_sum += fitness
            if current_sum >= roulette_wheel:
                selected_parents.append(population[i])
                break

    return selected_parents

In [13]:
def exclusion_selection(all_individuals, pop_size):
    all_individuals.sort(key=lambda x: sambrero_funk(x[0], x[1]), reverse=True)

    new_population = []
    included_chromosomes = {}

    for individual in all_individuals:
        chromosome = tuple(individual)

        if chromosome not in included_chromosomes:
            new_population.append(individual)
            included_chromosomes[chromosome] = True

            if len(new_population) == pop_size:
                break

    return new_population